# Практика
Используемые библиотеки

In [143]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [192]:
# url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = requests.get(url)
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df = pd.read_csv("data-20160608T1215-structure-20160608T1215.csv", sep=',', index_col='Показатель')
df['limit'] = df['Норматив'].str.replace(',', '.')
df['limit'] = df['limit'].str.findall(r'\d*[.,-]?\d+').explode()
df['result'] = df['Результат анализа'].str.replace(r'[а-яё]+[/][а-яё]+', '0')

#формируем границы интервалов
df['limit_a'] = [elem.split('-')[0] if '-' in elem else elem for elem in df['limit']]

df['limit_b'] = [elem.split('-')[1] if '-' in elem and not df['Норматив'].iloc[i].startswith('не более')
                                    else elem.split('-')[0]
                                                            for i, elem in enumerate(df['limit'])]


df[['limit_a', 'limit_b', 'result']] = df[['limit_a', 'limit_b', 'result']].astype('float64')
# df.drop(columns=['limit', 'Результат анализа'], inplace=True)

# Исследуем результат анализа на вхождение в интервал нормы
df_norma = df.query('(result >= limit_a and result <= limit_b) or\
                     (result <= limit_a and limit_a == limit_b)')
# Формируем поле вывода
res_positiv = 'Показатель в норме'
res_negative = 'Показатель выходит за пределы нормы'
df['Вывод'] =  [res_positiv 
                           if index in df_norma.index.values
                           else res_negative
                                             for index in df.index.values]

df

,Единица измерений,Результат анализа,Норматив,limit,result,limit_a,limit_b,Вывод
Показатель,,,,,,,,
pH,единицы pH,8.4,в пределах 6-9,6-9,8.400,6.0,9.0,Показатель в норме
Запах,баллы,1,не более 2-3,2-3,1.000,2.0,2.0,Показатель в норме
Цветность,градусы,б/цвета,не более 30,30,0.000,30.0,30.0,Показатель в норме
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,7-10,9.200,7.0,10.0,Показатель в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",1.5,0.420,1.5,1.5,Показатель в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",3.3,0.017,3.3,3.3,Показатель в норме
Нитраты (по NO3),мг/дм3,24,не более 45,45,24.000,45.0,45.0,Показатель в норме
Фосфаты (P),мг/дм3,0.36,"не более 3,5",3.5,0.360,3.5,3.5,Показатель в норме
Хлориды (Cl),мг/дм3,200,не более 350,350,200.000,350.0,350.0,Показатель в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [250]:
# Ваше решение: 
n = 100000
box = ['Orange']*5 +['Apple']*4
# exp = 

# эксперимент - вытаскиваем 3 фрукта из ящика n раз
df = pd.DataFrame(map(lambda a: np.random.choice(box, size=3, replace=False), range(n)),
                      columns=['f1', 'f2', 'f3'])

A1 = np.sum((df.f1 == df.f2) & (df.f2 == df.f3)  & (df.f1 == 'Orange')) / n
print(f'Вероятность, что все три фрукта – апельсины - {A1:.3f}')

df
# Вывод: аналитический ответ=0.119 при количестве экспериментов = 100тыс ПОДТВЕРЖДАЕТСЯ.

Вероятность, что все три фрукта – апельсины - 0.119


,f1,f2,f3
0,Orange,Orange,Apple
1,Apple,Orange,Apple
2,Apple,Orange,Apple
3,Orange,Apple,Apple
4,Apple,Apple,Apple
...,...,...,...
99995,Orange,Apple,Orange
99996,Orange,Orange,Apple
99997,Apple,Apple,Apple
99998,Orange,Orange,Apple


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [249]:
# Ваше решение
elements = ['st']*7 + ['non-st']*3
member1 = np.random.choice(elements, size=3, replace=False)

# эксперимент - вытаскиваем 3 фрукта из ящика n раз
df = pd.DataFrame(map(lambda a: np.random.choice(elements, size=3, replace=False), range(n)), columns=['d1', 'd2', 'd3'])

A2 = np.sum((df.d1 == 'non-st') & (df.d1 == 'non-st') & (df.d3 == 'st')) / n

print(f'Вероятность, что мастер проверит ровно две детали - {A2:.3f}')

df

# Вывод - экспериментально аналитический ответ подтверждается.

Вероятность, что мастер проверит ровно две детали - 0.234


,d1,d2,d3
0,st,st,st
1,non-st,non-st,st
2,st,non-st,st
3,st,st,non-st
4,non-st,non-st,st
...,...,...,...
99995,non-st,non-st,st
99996,non-st,non-st,st
99997,non-st,st,st
99998,st,st,st
